# Constraint satisfaction problem

In the previous tasks the state was opaque: an algorithm could execute an action and transform a state to another state, but that was it. It could not depend in its search on the details of a state, because the state representation was different from problem to problem. In CSP this is no longer true: the state always consists of an assignment of values over some domains to variables. This enables an algorithm to interact with the state directly, not only through opaque actions. 

Allowed assignments are limited by constraints in the problem and broadly speaking they can be divided into three groups:

* *Unary constraints* each concerning only a single variable (e.g., $x>0$)
* *Binary constraints* relating two variables to each other (e.g., $x>y$)
* *General constraints* concerning more than two variables at once

In general, the shape of constraints is unrestricted and we are not limited to numeric domains and constraints in the form of inequalities. In principle, one could employ arbitrary boolean functions as constraints.

The following class `CSP` is a very limited representation of such a problem capable only of handling:

* finite domains that can be materialized (e.g., the set of all 32-bit integers is doable in theory; the set of all 64-bit integers is out of reach) 
* unary constraints by setting the domain of a variable appropriately;
* binary constraints defined in terms of Python's operators `==`, `!=`, `<`, `<=`, `>` and `>=` such that both sides are variables.

About its methods:

* The constructor (`__init__`) requires no arguments.
* `addVariable` requires two arguments: 
    * `var` - a variable (a string representing its name, but really, it could be any object);
    * `domain` - a collection of the allowed values for the variable
* `addBinaryConstraint` also requires three arguments:
    * `var1` - a variable, the left-hand side of the constraint
    * `op` - an operator (one of the following strings: `'=='`, `'!='`, `'<'`, `'<='`, `'>'`, `'>='`)
    * `var2` - a variable, the right-hand side of the constraint
* `verify` is used to verify whether the relation given by the second argument `op` holds between the values given as the first argument `left` and the third argument `right`. Returns `True` if it does and `False` otherwise.
* `is_complete` gets a single argument: a dictionary representing an assignment of values to variables. From now on, we call such a dictionary an *assignment*. It returns `True` if all the variables of the problem are present in the dictionary, and `False` otherwise.
* `is_consistent` also requries an assignment. It returns `True` if all the values in the assignment are such that they come from the respecitve domains, and no binary constrain is violated (but if the assignment is not complete then some may be unfulfilled).

In [24]:
class CSP:
    def __init__(self):
        self.domains = {}
        self.binary = []
    
    def addVariable(self, var, domain):        
        assert var not in self.domains
        self.domains[var] = set(domain)
        
    def addBinaryConstraint(self, var1, operator, var2):
        assert var1 in self.domains
        assert var2 in self.domains
        c = (var1, operator, var2)
        self.binary.append(c)      
        
    def verify(self, left, op, right):
        if op[0] == '=':
            return left == right
        if op == '!=':
            return left != right
        if op == '<':
            return left < right
        if op == '<=':
            return left <= right
        if op == '>':
            return left > right
        if op == '>=':
            return left >= right
        
    def is_complete(self, assignment):
        return self.domains.keys() <= assignment.keys() 
        
    def is_consistent(self, assignment):
        for var, value in assignment.items():            
            if value not in self.domains[var]:
                print("HOW???")
                return False
        for var1, op, var2 in self.binary:
            if var1 in assignment and var2 in assignment:
                if not self.verify(assignment[var1], op, assignment[var2]):
                    return False
        return True

## Making Australia colorful again

Lets model a map coloring problem. In particular, we'll color the following map of Australia. It can be simplified to a graph presented in the right-hand side of the picture below.

![](aima-images/fig_6_1.png)

We begin by creating a new problem in the variable `australia` over the domain consisting of three values: `'R'` statnding for the color red, `'G'` standing for green and `'B'` standing for blue. Of course the particular symbols used are not relevant. There are 7 teritories, so we add 7 variables to the problem.

In [25]:
australia = CSP()
australia.addVariable('WA', {'R', 'G', 'B'})
australia.addVariable('NT', {'R', 'G', 'B'})
australia.addVariable('Q', {'R', 'G', 'B'})
australia.addVariable('SA', {'R', 'G', 'B'})
australia.addVariable('NSW', {'R', 'G', 'B'})
australia.addVariable('V', {'R', 'G', 'B'})
australia.addVariable('T', {'R', 'G', 'B'})

Any two teritories of Australia sharing a border should be colored using different colours. We thus define 9 binary constraints.

In [26]:
australia.addBinaryConstraint('WA', '!=', 'NT')
australia.addBinaryConstraint('WA', '!=', 'SA')
australia.addBinaryConstraint('NT', '!=', 'SA')
australia.addBinaryConstraint('NT', '!=', 'Q')
australia.addBinaryConstraint('SA', '!=', 'Q')
australia.addBinaryConstraint('SA', '!=', 'NSW')
australia.addBinaryConstraint('SA', '!=', 'V')
australia.addBinaryConstraint('Q', '!=', 'NSW')
australia.addBinaryConstraint('NSW', '!=', 'V')

Let's see how the `verify` method works.

In [27]:
print("Is G=G?", australia.verify('G', '==', 'G'))
print("Is 5<7?", australia.verify(5, '<', 7))
print("Is 5>=7?", australia.verify(5, '>=', 7))

Is G=G? True
Is 5<7? True
Is 5>=7? False


Lets now consider an assignment such that *Western Australia* is red and the rest is unspecified. Is it a consistent assignment?

In [28]:
australia.is_consistent({'WA': 'R'})

True

Is it a complete assignment?

In [29]:
australia.is_complete({'WA': 'R'})

False

As expected it didn't work: there are still 9 unfulfiled constraints. So lets extend the solution.

In [30]:
assignment = {'WA': 'R', 'Q': 'R', 'V': 'R', 'NT': 'B', 'NSW': 'B', 'SA': 'G'}
print("Is consistent?", australia.is_consistent(assignment))
print("Is complete?", australia.is_complete(assignment))

Is consistent? True
Is complete? False


Now it almost works: all the constraints are satisfied, but the solution is still not complete, as Tasmania is still colorless.

In [31]:
assignment = {'WA': 'R', 'Q': 'R', 'V': 'R', 'NT': 'B', 'NSW': 'B', 'SA': 'G', 'T': 'B'}
print("Is consistent?", australia.is_consistent(assignment))
print("Is complete?", australia.is_complete(assignment))

Is consistent? True
Is complete? True


Finally, we have manually collored the map of Australia! Now let's make computer do this for us.

## Task 1: Backtracking

Complete the following cell of code by implementing the recursive backtracking algorithm. Basically, it is a recursive DFS where an action is assigning a value to a variable. Avoid making copies of assignments, instead operate on a single assignment, and update it accordingly. Increment `self.counter` every time you check wheter an assigment is a solution (i.e., is consistent and complete). Use the provided methods `select_unassigned_variable` and `order_domain_values` to, respectively, get a next variable to assign a value to, and to get the list of allowed values for the variable. `solve` should return a solution (i.e., a complete, consistent assignment) or `None`.

In [32]:
class RecursiveBacktracking:
    def __init__(self, csp: CSP):
        self.csp = csp
        self.counter = 0
        
    def select_unassigned_variable(self, assignment):
        for var in self.csp.domains.keys():
            if var not in assignment:
                return var
        return None
    
    def order_domain_values(self, variable, assignment):
        return self.csp.domains[variable]
        
    
    def solve(self):
        self.counter = 0
        result = self.backtrack({})
        return result

    def backtrack(self, assignment):
        # self.counter+=1
        if self.csp.is_complete(assignment):
            return assignment
        n = self.select_unassigned_variable(assignment)
        possible = self.order_domain_values(n,assignment)
        for p in possible:
            assignment[n] = p
            self.counter+=1
            if self.csp.is_consistent(assignment):
                result = self.backtrack(assignment)
                if result!=False:
                    return result
                assignment.pop(n)
        return False
        

Let's test it on the Australia problem! In the cell below you should get a consistent, complete assignment, i.e., a solution to the problem.

In [33]:
solver = RecursiveBacktracking(australia)
assignment = solver.solve()
print("Assignment", assignment)
print("Is consistent?", australia.is_consistent(assignment))
print("Is complete?", australia.is_complete(assignment))
print("# considered assignments", solver.counter)

Assignment {'WA': 'B', 'NT': 'R', 'Q': 'B', 'SA': 'G', 'NSW': 'R', 'V': 'B', 'T': 'B'}
Is consistent? True
Is complete? True
# considered assignments 11


## Task 2: Extend `RecursiveBacktracking` with the MRV and degree heuristics

Modify your `RecursiveBacktracking` to include the MRV (minimum remaining values) heuristics and the degree heuristics. Implement them in the appropriate places below in the class `RecursiveBacktrackingWithHeuristics`. If you want, you can also implement the least constraining value heuristics, but this is not mandatory. Follow the same assumptions as for `RecursiveBacktracking`, so expose the main backtracking algorithm as the `solve` and keep track of the number of considered assignments in the `counter` variable.

In [34]:
class RecursiveBacktrackingWithHeuristics:
    def __init__(self, csp: CSP):
        self.csp = csp
        self.counter = 0

    def select_unassigned_variable(self, assignment):
        available = {}
        constraints = {}
        for var in self.csp.domains.keys():
            if var not in assignment:
                # get variable with fewest options -> MRV
                available[var] = self.order_domain_values(var, assignment)
                # get variable with most constraints -> DEGREE
                constraints[var] = sum(
                    [-1 if var in t else 0 for t in self.csp.binary])
        for var in sorted(available, key=lambda x: (len(available[x]), constraints[x])):
            return var
        print("???")
        return None

    def order_domain_values(self, variable, assignment):
        available = self.csp.domains[variable].copy()
        for var1, op, var2 in self.csp.binary:
            if var1 == variable and var2 in assignment:
                if op == '!=':
                    if assignment[var2] in available:
                        available.remove(assignment[var2])
            if var2 == variable and var1 in assignment:
                if op == '!=':
                    if assignment[var1] in available:
                        available.remove(assignment[var1])
        return available

    def solve(self):
        self.counter = 0
        assignment = {}
        result = self.backtrack(assignment)
        return result

    def backtrack(self, assignment):
        if self.csp.is_complete(assignment):
            return assignment
        n = self.select_unassigned_variable(assignment)
        possible = self.order_domain_values(n, assignment)
        for p in possible:
            assignment[n] = p
            self.counter += 1
            if self.csp.is_consistent(assignment):
                result = self.backtrack(assignment)
                if result!=False:
                    return result
                assignment.pop(n)
        return False

Lets test your implementation by solving the problem of coloring Australia!

In [35]:
solver = RecursiveBacktrackingWithHeuristics(australia)
assignment = solver.solve()
print("Assignment", assignment)
print("Is consistent?", australia.is_consistent(assignment))
print("Is complete?", australia.is_complete(assignment))
print("# considered assignments", solver.counter)

Assignment {'SA': 'B', 'NT': 'R', 'Q': 'G', 'NSW': 'R', 'WA': 'G', 'V': 'G', 'T': 'B'}
Is consistent? True
Is complete? True
# considered assignments 7


**Compare the number of considered assignments for `RecursiveBacktracking` and `RecursiveBacktrackingWithHeuristics`. How do they differ? Why do you think is that?**

Using heuristics, we make a more informed choice while choosing the next step, which leads to fewer mistakes and backtracking, allowing for fewer checks of the `is_consistent()` and `is_complete()` methods. 

## Sudoku!

Coloring Australia is really a bit boring. Lets deal with something more exciting: SUDOKU! I assume you know the rules, but if you don't, go look them up, spend some time solving a few puzzles and familiarize yourself with the puzzle.
The cell below defines a string `puzzle` which represents a Sudoku puzzle from the picture below. A digit in `puzzle` corresponds to a digit in the puzzle, while an underscore `_` corresponds to a blank cell in the puzzle. For your convenience there's also variable `solution` containing the correct solution to this particular puzzle.

![](aima-images/fig_6_4.png)

In [36]:
puzzle = '''
__3_2_6__
9__3_5__1
__18_64__
__81_29__
7_______8
__67_82__
__26_95__
8__2_3__9
__5_1_3__
'''

In [37]:
# puzzle = '''
# 8________
# __36_____
# _7__9_2__
# _5___7___
# ____457__
# ___1___3_
# __1____68
# __85___1_
# _9____4__
# '''

In [38]:
solution = '''
483921657
967345821
251876493
548132976
729564138
136798245
372689514
814253769
695417382
'''

## Task 3: Sudoku as CSP

Using the string `puzzle` defined above write in the cell below code that will add all the necessary constraints to the CSP represented by the variable `sudoku`. If you can and want, write a parser that will construct the CSP from a string representation of the puzzle. Otherwise, you can construct the problem manually.

In [39]:
sudoku = CSP()
p = puzzle.split()
for i in range(9):
    for j in range(9):
        if p[i][j]!='_':
            sudoku.addVariable((i,j),{int(p[i][j])})
            continue
        sudoku.addVariable((i,j),range(1,10))

for i in range(3):
    for j in range(3):
        for l in range(9):
            for m in range(9):
                if l>m:
                    t1 = l//3+i*3,l%3+j*3
                    t2 = m//3+i*3,m%3+j*3
                    sudoku.addBinaryConstraint(t1,'!=',t2)
for i in range(9):
    for j in range(9):
        for k in range(j,9):
            if k in range(j//3*3,j//3*3+3):
                continue
            sudoku.addBinaryConstraint((i,j),'!=',(i,k))
for i in range(9):
    for j in range(9):
        for k in range(i,9):
            if k in range(i//3*3,i//3*3+3):
                continue
            sudoku.addBinaryConstraint((i,j),'!=',(k,j))


Let's see if it works... Unless you did some clever tricks and/or are quite patient - it does not, it is too slow.

In [40]:
solver = RecursiveBacktrackingWithHeuristics(sudoku)
assignment = solver.solve()
display = [[0 for i in range(9)] for j in range(9)]
for i,j in assignment:
    display[i][j] = assignment[(i,j)]
from pprint import pprint

print("Assignment")
pprint(display)
print("Is consistent?", sudoku.is_consistent(assignment))
print("Is complete?", sudoku.is_complete(assignment))
print("# considered assignments", solver.counter) 

Assignment
[[4, 8, 3, 9, 2, 1, 6, 5, 7],
 [9, 6, 7, 3, 4, 5, 8, 2, 1],
 [2, 5, 1, 8, 7, 6, 4, 9, 3],
 [5, 4, 8, 1, 3, 2, 9, 7, 6],
 [7, 2, 9, 5, 6, 4, 1, 3, 8],
 [1, 3, 6, 7, 9, 8, 2, 4, 5],
 [3, 7, 2, 6, 8, 9, 5, 1, 4],
 [8, 1, 4, 2, 5, 3, 7, 6, 9],
 [6, 9, 5, 4, 1, 7, 3, 8, 2]]
Is consistent? True
Is complete? True
# considered assignments 81


## Task 4: Solving sudoku

In order to make it working, we need to extend backtracking with inference. Complete the following class `RecursiveBacktrackingWithAC3` in such a way that it uses the same heuristics as `RecursiveBacktrackingWithHeuristics` and on top of that uses the AC3 algorithm after a new value is added to the assignment.

In this task, you don't need to stick to the previous API with method. A constructor with a single argument, and an argumentless `solve` are sufficient, heuristics and AC3 can be handled however you want.

In [41]:
from queue import Queue
from copy import deepcopy
class RecursiveBacktrackingWithAC3:
    def __init__(self, csp: CSP):
        self.csp = csp
        self.counter = 0
        self.domains = deepcopy(csp.domains)

    def ac3(self):
        queue = Queue()
        for arc in self.csp.binary:
            queue.put((arc[0],arc[2]))
        while not queue.empty():
            xi,xj = queue.get()
            if xi==xj:
                print("Same",xi,xj)
            if self.remove_inconsistant_variables(xi,xj):
                    for x in self.neighbours(xi):
                        queue.put((x,xi))
            # break

    def remove_inconsistant_variables(self,xi,xj):
        removed = False
        to_remove = set()
        for x in self.domains[xi]:
            satisfies = False
            for y in self.domains[xj]:
                if x!=y:
                    satisfies=True
                    break
            if not satisfies:
                to_remove.add(x)
                removed = True
        for i in to_remove:
            self.domains[xi].remove(i)
        return removed


    def neighbours(self, x):
        neighbours = []
        for t1, op, t2 in self.csp.binary:
            if x==t1:
                neighbours.append(t2)
            if x==t2:
                neighbours.append(t1)
        return neighbours


    def select_unassigned_variable(self, assignment):
        available = {}
        constraints = {}
        for var in self.domains.keys():
            if var not in assignment:
                # get variable with fewest options -> MRV
                available[var] = self.order_domain_values(var, assignment)
                # get variable with most constraints -> DEGREE
                constraints[var] = sum(
                    [-1 if var in t else 0 for t in self.csp.binary])

        for var in sorted(available, key=lambda x: (len(available[x]), constraints[x])):
            return var
        print("???")
        return None

    def order_domain_values(self, variable, assignment):
        available = self.domains[variable].copy()
        for var1, op, var2 in self.csp.binary:
            if var1 == variable and var2 in assignment:
                if op == '!=':
                    if assignment[var2] in available:
                        available.remove(assignment[var2])
            if var2 == variable and var1 in assignment:
                if op == '!=':
                    if assignment[var1] in available:
                        available.remove(assignment[var1])
        return available

    def solve(self):
        self.counter = 0
        assignment = {}
        self.ac3()
        result = self.backtrack(assignment)
        return result

    def backtrack(self, assignment):
        # self.counter += 1
        if self.csp.is_complete(assignment):
            return assignment
        n = self.select_unassigned_variable(assignment)
        possible = self.order_domain_values(n, assignment)
        for p in possible:
            assignment[n] = p
            # display = [[0 for i in range(9)] for j in range(9)]
            # for i,j in assignment:
            #     display[i][j] = assignment[(i,j)]
            # pprint(display)
            # print(i,j)
            self.counter += 1
            if self.csp.is_consistent(assignment):
                result = self.backtrack(assignment)
                if result!=False:
                    return result
                assignment.pop(n)
        return False



Let's test it on both considered problem. If everything went all right, both cells should terminate within few seconds at most, and you should see that the number of considered assignments is very small.

In [42]:
solver = RecursiveBacktrackingWithAC3(australia)
assignment = solver.solve()
print("Assignment", assignment)
print("Is consistent?", australia.is_consistent(assignment))
print("Is complete?", australia.is_complete(assignment))
print("# considered assignments", solver.counter)

Assignment {'SA': 'B', 'NT': 'G', 'Q': 'R', 'NSW': 'G', 'WA': 'R', 'V': 'R', 'T': 'B'}
Is consistent? True
Is complete? True
# considered assignments 7


In [43]:
solver = RecursiveBacktrackingWithAC3(sudoku)
assignment = solver.solve()
display = [[0 for i in range(9)] for j in range(9)]
for i,j in assignment:
    display[i][j] = assignment[(i,j)]
pprint(display)

# print("Assignment", assignment)
print("Is consistent?", sudoku.is_consistent(assignment))
print("Is complete?", sudoku.is_complete(assignment))
print("# considered assignments", solver.counter)

[[4, 8, 3, 9, 2, 1, 6, 5, 7],
 [9, 6, 7, 3, 4, 5, 8, 2, 1],
 [2, 5, 1, 8, 7, 6, 4, 9, 3],
 [5, 4, 8, 1, 3, 2, 9, 7, 6],
 [7, 2, 9, 5, 6, 4, 1, 3, 8],
 [1, 3, 6, 7, 9, 8, 2, 4, 5],
 [3, 7, 2, 6, 8, 9, 5, 1, 4],
 [8, 1, 4, 2, 5, 3, 7, 6, 9],
 [6, 9, 5, 4, 1, 7, 3, 8, 2]]
Is consistent? True
Is complete? True
# considered assignments 81


------------
The pictures are from "Artificial Intelligence: A Modern Approach" 3rd ed.